In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.vecm import VECM, select_order
from statsmodels.tools.eval_measures import rmse
import matplotlib.pyplot as plt
from Codigo import *
import seaborn as sn
from statsmodels.stats.diagnostic import normal_ad
sn.set(style="whitegrid")

### Cargar Datos

In [3]:
Datos = pd.read_excel('../Db.xlsx', sheet_name= 'VARIABLES DEL EXAMEN')
Datos = Datos.T
Datos.columns = Datos.iloc[1]
Datos = Datos[2:]
Datos.index = pd.date_range(start='1961-12-31', periods=len(Datos), freq='Y')
data = Datos.copy()

In [8]:
data.columns

Index(['Tasa de mortalidad, bebés (por cada 1.000 nacidos vivos)',
       'Activos exteriores netos (UMN a precios actuales)',
       'Agricultura, valor agregado (% del crecimiento anual)',
       'Agricultura, valor agregado (UMN: Unidades de moneda nacional, a precios constantes)',
       'Agricultura, valor agregado (US$ a precios constantes de 2010)',
       'Ahorro interno bruto (US$ a precios actuales)',
       'Ajuste de los términos de intercambio (UMN a precios constantes)',
       'AOD neta recibida (US$ constantes de 2014)',
       'Área de tierra (kilómetros cuadrados)',
       'Asistencia oficial para el desarrollo (AOD) neta y ayuda oficial neta recibidas (US$ constantes de 2013)',
       ...
       'Tierras agrícolas (% del área de tierra)',
       'Tierras agrícolas (kilómetros cuadrados)',
       'Tierras cultivables (% del área de tierra)',
       'Tierras cultivables (hectáreas por persona)',
       'Tierras cultivables (hectáreas)',
       'Tierras destinadas al cu

In [13]:
Columnas(data,'int')

Ahorro interno bruto (US$ a precios actuales)
Ajuste de los términos de intercambio (UMN a precios constantes)
Crédito interno al sector privado otorgado por los bancos (% del PIB)
Crédito interno neto (UMN a precios actuales)
Ingreso interno bruto (UMN a precios constantes)
Recursos de agua dulce  internos renovables per cápita (metros cúbicos)
Recursos de agua dulce  internos renovables, total (billones de metros cúbicos)


In [ ]:
data = data[['Total de reservas (incluye oro, US$ a precios actuales)','Ahorro interno bruto (US$ a precios actuales)','Crédito interno al sector privado otorgado por los bancos (% del PIB)']]
data.columns = ['Reservas','SIB','Credito Bancario']

### Analisis de Estacionariedad a las Variables

In [4]:
DFuller(data['PIB'])
DFuller(data['Gasto'])
DFuller(data['Inversión'])

KeyError: 'PIB'

### Analisis de Cointegración

In [13]:
print('Al 95% de confianza')
Johansen_Test(data,1,10,1)

Al 95% de confianza
Matriz de Valores Criticos vs Maximo Valor Propio
            cvm     eigv        Hipotesis
r = 0     15.0006    72.42    Se rechaza r = 0, existe mas de 0 relación/nes de cointegración
r = 1     17.1481    19.6595    Se rechaza r = 1, existe mas de 1 relación/nes de cointegración
r = 2     21.7465    0.2433    No hay suficiente evidencia para rechazar la hipótesis nula de r = 2.
Tabla de Valores Criticos vs Traza Estadistica
            cvt     TS          Hipotesis
r = 0     16.1619    92.3228    Se rechaza r = 0, existe mas de 0 relación/nes de cointegración
r = 1     18.3985    19.9028    Se rechaza r = 1, existe mas de 1 relación/nes de cointegración
r = 2     23.1485    0.2433    No hay suficiente evidencia para rechazar la hipótesis nula de r = 2.


### Modelo Inicial

In [5]:

lag_order = select_order(data, maxlags= 10, deterministic= 'n', seasons= 0)

print(lag_order)

<statsmodels.tsa.vector_ar.var_model.LagOrderResults object. Selected orders are: AIC -> 10, BIC -> 0, FPE -> 10, HQIC ->  10>


Para seleccionar el orden el sistema nos dice que para aic y hqic es 10 por lo que usaremos ese 

In [6]:
model = VECM(data ,coint_rank= 2, k_ar_diff= 10)
result = model.fit()
alpha_matrix = result.alpha
print("Matriz de Coeficientes de Corrección de Errores (alpha):")
print(alpha_matrix)


Matriz de Coeficientes de Corrección de Errores (alpha):
[[-0.11692322  1.38542498]
 [-0.03631507  0.08667444]
 [-0.00896661  0.45129364]]


In [7]:
print(result.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation PIB
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
L1.PIB           -0.9141      0.235     -3.893      0.000      -1.374      -0.454
L1.Inversión      0.1951      0.698      0.280      0.780      -1.172       1.563
L1.Gasto          0.8174      0.409      1.998      0.046       0.015       1.619
L2.PIB           -1.4526      0.264     -5.499      0.000      -1.970      -0.935
L2.Inversión     -0.8823      0.651     -1.354      0.176      -2.159       0.394
L2.Gasto          1.2191      0.408      2.984      0.003       0.419       2.020
L3.PIB           -1.1923      0.333     -3.585      0.000      -1.844      -0.540
L3.Inversión      2.5513      0.882      2.893      0.004       0.823       4.280
L3.Gasto          0.0258      0.484      0.053      0.958      -0.924       0.975
L4.PIB         

In [8]:
# Paso 6: Análisis de Impulsos-Respuestas
irf = result.irf(periods=10)
irf.plot()
plt.show()

In [9]:
result.plot_forecast(steps = 10)

In [10]:
steps = 10
forecast, lower, upper = result.predict(steps, 0.05)
Fecha = pd.date_range(start=data.index[-1], periods=steps, freq='Y')
for i in range(len(data.columns)):
    t_pred = forecast[:,i]
    plt.plot(Fecha, t_pred, linestyle = '--')
plt.legend()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [11]:

# Obtén los residuos
residuals = result.resid

# Realiza una prueba de normalidad de los residuos (por ejemplo, prueba Anderson-Darling)
ad_statistic, critical_values = normal_ad(residuals)

# Compara el estadístico con los valores críticos
for i in range(len(ad_statistic)):
    print(f'Anderson-Darling statistic: {ad_statistic[i]}, Critical value: {critical_values[i]}') # type: ignore

    # Compara el estadístico con el valor crítico
    if ad_statistic[i] > critical_values[i]: # type: ignore
        print(f'Se rechaza la hipótesis nula de normalidad para el estadístico {i + 1}.')
    else:
        print(f'No hay suficiente evidencia para rechazar la hipótesis nula de normalidad para el estadístico {i + 1}.')

Anderson-Darling statistic: 0.42950781364649515, Critical value: 0.29530691941167625
Se rechaza la hipótesis nula de normalidad para el estadístico 1.
Anderson-Darling statistic: 0.21621636632591645, Critical value: 0.834384533656835
No hay suficiente evidencia para rechazar la hipótesis nula de normalidad para el estadístico 2.
Anderson-Darling statistic: 0.2967038449269239, Critical value: 0.5758604374522709
No hay suficiente evidencia para rechazar la hipótesis nula de normalidad para el estadístico 3.
